In [ ]:
# Clean + pin versions that play nice together
!pip -q uninstall -y numpy torch torchvision torchaudio open3d || true
!pip -q install "numpy<2" open3d==0.19.0
!pip -q install torch==2.2.2+cu121 torchvision==0.17.2+cu121 torchaudio==2.2.2+cu121 \
  --index-url https://download.pytorch.org/whl/cu121


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.9.0 requires torch>=2.0.0, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, which is not installed.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
sentence-transformers 5.0.0 requires torch>=1.11.0, which is not installed.
peft 0.17.0 requires torch>=1.13.0, which is not installed.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# TODO: set this to the folder that DIRECTLY contains Paris.ply, Lille1.ply, Lille2.ply
DATASET_DIR = "/content/drive/MyDrive/Benchmark/raw_data/Lille2"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls -lh "$DATASET_DIR"


total 4.0K
drwx------ 2 root root 4.0K Aug 10 21:21 file


In [ ]:
# Example: save weights in /content (you can put them in Drive too)
!wget -O /content/randlanet_pl3d.pth \
  https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_parislille3d_202201071330utc.pth


--2025-08-10 21:40:29--  https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_parislille3d_202201071330utc.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20170579 (19M) [application/octet-stream]
Saving to: ‘/content/randlanet_pl3d.pth’

/content/randlanet_ 100%[===================>]  19.24M  --.-KB/s    in 0.09s   

2025-08-10 21:40:29 (209 MB/s) - ‘/content/randlanet_pl3d.pth’ saved [20170579/20170579]



In [ ]:
!git clone --depth 1 https://github.com/isl-org/Open3D-ML.git
%cd Open3D-ML


fatal: destination path 'Open3D-ML' already exists and is not an empty directory.
/content/Open3D-ML


In [ ]:
import os

DATASET_DIR = "/content/drive/MyDrive/Benchmark/raw_data/Lille2"

In [ ]:
# --- Choose a tile and copy it locally as "Paris.ply"
# Assuming Lille_11.ply is the file you want to copy, located in the 'file' subdirectory in your Drive
TILE="/content/drive/MyDrive/Benchmark/raw_data/Lille2/file/Lille_11.ply"   # <-- change to 12/13 if you prefer
# Assuming your checkpoint file is in the root of your Drive, update if needed
CKPT_PATH="/content/drive/MyDrive/randlanet_pl3d.pth"                        # <-- must be a .pth file

# Create the destination directory if it doesn't exist
!mkdir -p /content/pl3d_single
# Use the ! prefix to run cp as a shell command
!cp "$TILE" /content/pl3d_single/Paris.ply
!ls -lh /content/pl3d_single

total 668M
-rw------- 1 root root 668M Aug 10 21:47 Paris.ply


In [ ]:
!find /content -maxdepth 5 -path "*/logs/*" -type d -printf "%p\n"
!find /content -maxdepth 6 -path "*/logs/*" -type f -printf "%p\n" | tail -n 30


/content/.config/logs/2025.08.07
/content/drive/MyDrive/Open3D-ML_backup/logs/RandLANet_ParisLille3D_torch
/content/drive/MyDrive/Open3D-ML_backup/logs/cache
/content/Open3D-ML/logs/cache
/content/Open3D-ML/logs/cache/be9a7263aff0fda39db854e88d6a8f25
/content/Open3D-ML/logs/RandLANet_ParisLille3D_torch
/content/Open3D-ML/logs/RandLANet_ParisLille3D_torch/checkpoint
/content/Open3D-ML/.git/logs/refs
/content/Open3D-ML/.git/logs/refs/remotes
/content/Open3D-ML/.git/logs/refs/heads
/content/.config/logs/2025.08.07/13.42.07.355819.log
/content/.config/logs/2025.08.07/13.42.16.048540.log
/content/.config/logs/2025.08.07/13.42.25.647837.log
/content/.config/logs/2025.08.07/13.41.46.739789.log
/content/.config/logs/2025.08.07/13.42.17.248935.log
/content/.config/logs/2025.08.07/13.42.26.351069.log
/content/drive/MyDrive/Open3D-ML_backup/.git/logs/HEAD
/content/drive/MyDrive/Open3D-ML_backup/logs/RandLANet_ParisLille3D_torch/log_test_2025-08-10_20-54-30.txt
/content/drive/MyDrive/Open3D-ML_bac

In [ ]:
import glob, os

DATASET_DIR = "/content/drive/MyDrive/Benchmark/raw_data"  # parent of training_10_classes/, test_10_classes/

# Build lists relative to DATASET_DIR
train_all = [os.path.relpath(p, DATASET_DIR)
             for p in sorted(glob.glob(os.path.join(DATASET_DIR, "training_10_classes/*.ply")))]

# Simple split (e.g., last 5 files as val) — tweak as you like
val_files   = train_all[-5:] if len(train_all) > 5 else train_all[:1]
train_files = [p for p in train_all if p not in val_files]

test_files  = [os.path.relpath(p, DATASET_DIR)
               for p in sorted(glob.glob(os.path.join(DATASET_DIR, "test_10_classes/*.ply")))]

cfg.dataset.update({
    "name": "ParisLille3D",
    "dataset_path": DATASET_DIR,
    "train_files": train_files,
    "val_files":   val_files,
    "test_files":  test_files,  # usually unlabeled → inference-only
})
cfg.model["num_classes"] = 10


In [ ]:
import os, torch
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

# --- paths ---
# DATASET_DIR should be the folder that CONTAINS the 'file' subdirectory
DATASET_DIR = "/content/drive/MyDrive/Benchmark/raw_data/Lille2"  # contains a 'file/' subdir
# Assuming your checkpoint file is in the root of your Drive
CKPT_PATH   = "/content/drive/MyDrive/randlanet_pl3d.pth"
# Assuming your Open3D-ML backup with configs is here
CONFIG_PATH = "/content/drive/MyDrive/Open3D-ML_backup/Open3D-ML/ml3d/configs/randlanet_parislille3d.yml"


assert os.path.isfile(CONFIG_PATH), f"Config missing: {CONFIG_PATH}"
cfg = _ml3d.utils.Config.load_from_file(CONFIG_PATH)

# --- dataset/model/pipeline ---
# Make the config ParisLille3D-friendly with relative paths
cfg.dataset.update({
    "name": "ParisLille3D",
    "dataset_path": DATASET_DIR, # Directory containing the 'file' subdirectory
    "train_files": ["file/Lille1.ply"],   # relative to DATASET_DIR
    "val_files":   ["file/Lille2.ply"],
    "test_files":  ["file/Paris.ply"],    # relative to DATASET_DIR
})
# Set num_classes to 9 to match the checkpoint's output layer size
cfg.model["num_classes"] = 9


dataset = ml3d.datasets.ParisLille3D(**cfg.dataset)

device = "cpu" if torch.cuda.is_available() else "cpu" # Explicitly set device to cpu
print("Using device:", device)
print("CUDA available:", torch.cuda.is_available())

model = ml3d.models.RandLANet(**cfg.model)
pipeline = ml3d.pipelines.SemanticSegmentation(
    model=model, dataset=dataset, device=device, **cfg.pipeline
)

assert os.path.isfile(CKPT_PATH), f"Missing checkpoint: {CKPT_PATH}"
pipeline.load_ckpt(CKPT_PATH)

# --- evaluate on VAL (metrics only if labels exist) ---
val_list = dataset.get_split_list("val")
print("Resolved val files:", val_list) # Print resolved val files

if val_list:
    val_split = dataset.get_split("val")
    # Check if the val split is not empty before attempting to get data
    if val_split.get_split_list(): # Use get_split_list() on the split object to check if it has files
        sample = val_split.get_data(0)
        if "label" in sample:
            print("Evaluating on VAL (OA / per-class IoU / mIoU) ...")
            # run_test() evaluates the 'test' split, so reuse the same cfg but
            # point test_files to your val files for metrics
            eval_cfg = _ml3d.utils.Config.load_from_file(CONFIG_PATH) # Load config again for eval
            eval_cfg.dataset.update({
                "name": "ParisLille3D",
                "dataset_path": DATASET_DIR,
                "train_files": [], # Empty train for eval
                "val_files": [],   # Empty val for eval
                "test_files": cfg.dataset["val_files"], # Use val files as test files for evaluation
            })
            eval_cfg.model["num_classes"] = 9 # Ensure num_classes is set for eval model config

            eval_ds  = ml3d.datasets.ParisLille3D(**eval_cfg.dataset)
            eval_pl  = ml3d.pipelines.SemanticSegmentation(model=model, dataset=eval_ds, device=device, **eval_cfg.pipeline)

            eval_pl.load_ckpt(CKPT_PATH)
            eval_pl.run_test()
        else:
             print("Val files do not have labels. Skipping evaluation with metrics.")
    else:
        print("Val split is empty. Skipping evaluation.")

else:
    print("No val files resolved by the dataset. Skipping evaluation.")

# --- inference on TEST (usually unlabeled in PL3D benchmark) ---
test_list = dataset.get_split_list("test")
print("Resolved test files:", test_list) # Print resolved test files

if test_list:
    print("Running inference on TEST split ...")
    test_split = dataset.get_split("test")
    if test_split.get_split_list(): # Check if test split is not empty
        # Check if test files have labels before running test (for metrics)
        try:
            sample = test_split.get_data(0)
            if "label" in sample:
                 print("Test files have labels → running evaluation (OA / per-class IoU / mIoU)")
                 pipeline.run_test() # Use the main pipeline for test evaluation if labels exist
            else:
                 print("Test files do not have labels → running inference-only")
                 _ = pipeline.run_inference(test_split)   # predictions saved under ./logs by default
                 print("Inference complete.")
        except IndexError:
             print("IndexError: Could not get data from test split. This usually means the dataset did not load any files for the test split.")
        except Exception as e:
             print(f"An error occurred during test split processing: {e}")
    else:
        print("Test split is empty. Skipping inference/evaluation on test split.")
else:
    print("No test files resolved by the dataset. Skipping inference/evaluation on test split.")

Using device: cpu
CUDA available: False
Resolved val files: []
No val files resolved by the dataset. Skipping evaluation.
Resolved test files: []
No test files resolved by the dataset. Skipping inference/evaluation on test split.


In [ ]:
import os, glob

DATASET_DIR = "/content/drive/MyDrive/Benchmark/"

def rel_glob(root, *patterns):
    hits = []
    for pat in patterns:
        hits += glob.glob(os.path.join(root, pat), recursive=True)
    hits = sorted(set(hits))
    return [os.path.relpath(p, root) for p in hits]

# match both flat and nested layouts
train_all = rel_glob(DATASET_DIR,
                     "training_10_classes/*.ply",
                     "training_10_classes/**/*.ply")
test_files = rel_glob(DATASET_DIR,
                      "test_10_classes/*.ply",
                      "test_10_classes/**/*.ply")

# make a small val split from train (at least 1)
val_files   = train_all[-5:] if len(train_all) > 5 else train_all[:1]
train_files = [p for p in train_all if p not in val_files]

print(f"train: {len(train_files)}  val: {len(val_files)}  test: {len(test_files)}")
print("sample train:", train_files[:3])
print("sample val:", val_files[:3])
print("sample test:", test_files[:3])

# verify they really exist
for split, rels in [("train", train_files), ("val", val_files), ("test", test_files)]:
    for r in rels[:5]:
        print(split, "→", os.path.join(DATASET_DIR, r), os.path.isfile(os.path.join(DATASET_DIR, r)))


train: 3  val: 1  test: 3
sample train: ['training_10_classes/Lille1_2.ply', 'training_10_classes/Lille2.ply', 'training_10_classes/Paris.ply']
sample val: ['training_10_classes/Lille1_1.ply']
sample test: ['test_10_classes/ajaccio_2.ply', 'test_10_classes/ajaccio_57.ply', 'test_10_classes/dijon_9.ply']
train → /content/drive/MyDrive/Benchmark/training_10_classes/Lille1_2.ply True
train → /content/drive/MyDrive/Benchmark/training_10_classes/Lille2.ply True
train → /content/drive/MyDrive/Benchmark/training_10_classes/Paris.ply True
val → /content/drive/MyDrive/Benchmark/training_10_classes/Lille1_1.ply True
test → /content/drive/MyDrive/Benchmark/test_10_classes/ajaccio_2.ply True
test → /content/drive/MyDrive/Benchmark/test_10_classes/ajaccio_57.ply True
test → /content/drive/MyDrive/Benchmark/test_10_classes/dijon_9.ply True


In [ ]:
import os, torch
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

# --- use the lists you just built ---
# DATASET_DIR should be the folder that CONTAINS the 'file' subdirectory
DATASET_DIR = "/content/drive/MyDrive/Benchmark/"  # contains training_10_classes/ & test_10_classes/ -- Corrected path to match user's likely data location
# Assuming your Open3D-ML backup with configs is here
CONFIG_PATH = "/content/drive/MyDrive/Open3D-ML_backup/Open3D-ML/ml3d/configs/randlanet_parislille3d.yml"


assert os.path.isfile(CONFIG_PATH), f"Config missing: {CONFIG_PATH}"
cfg = _ml3d.utils.Config.load_from_file(CONFIG_PATH)

# Make the config ParisLille3D-friendly with relative paths
cfg.dataset.update({
    "name": "ParisLille3D",
    "dataset_path": DATASET_DIR, # Directory containing the 'file' subdirectory
    # Use relative paths to files within the 'file' subdirectory
    "train_files": ["training_10_classes/Lille1.ply"], # Example: assuming Lille1.ply is in file/
    "val_files":   ["training_10_classes/Lille1_2.ply"], # Example: assuming Lille2.ply is in file/
    "test_files":  ["test_10_classes/dijon_9.ply"],  # Example: assuming Paris.ply is in file/
})

# Set num_classes to 9 to match the checkpoint
cfg.model["num_classes"] = 9  # PL3D checkpoint has 9 classes

# (CPU-only tip) keep batches tiny
cfg.pipeline["batch_size"] = min(cfg.pipeline.get("batch_size", 1), 1)

# --- build dataset/model/pipeline ---
dataset = ml3d.datasets.ParisLille3D(**cfg.dataset)
print("Resolved -> train:", len(dataset.get_split_list("train")),
      "val:", len(dataset.get_split_list("val")),
      "test:", len(dataset.get_split_list("test")))

device = "cpu" if torch.cuda.is_available() else "cpu" # Explicitly set device to cpu
print("Using device:", device)
print("CUDA available:", torch.cuda.is_available())

model = ml3d.models.RandLANet(**cfg.model)
pipeline = ml3d.pipelines.SemanticSegmentation(
    model=model, dataset=dataset, device=device, **cfg.pipeline
)

CKPT_PATH = "/content/drive/MyDrive/randlanet_pl3d.pth" # Assuming your checkpoint is in the root of your Drive
assert os.path.isfile(CKPT_PATH), f"Missing checkpoint: {CKPT_PATH}"
pipeline.load_ckpt(CKPT_PATH)

# --- evaluate on VAL (metrics only if labels exist) ---
val_list = dataset.get_split_list("val")
if val_list:
    val_split = dataset.get_split("val")
    # Check if val split is not empty before getting data
    if val_split.get_split_list(): # Use get_split_list() on split object
        sample = val_split.get_data(0)
        if "label" in sample:
            print("Evaluating on VAL (OA / per-class IoU / mIoU) ...")
            # run_test() uses the dataset's 'test' split, so build a temp dataset with test=val
            eval_ds_cfg = dict(cfg.dataset)
            eval_ds_cfg["test_files"] = cfg.dataset["val_files"]
            eval_ds  = ml3d.datasets.ParisLille3D(**eval_ds_cfg)
            eval_pl  = ml3d.pipelines.SemanticSegmentation(model=model, dataset=eval_ds,
                                                           device=device, **cfg.pipeline)
            eval_pl.load_ckpt(CKPT_PATH)
            eval_pl.run_test()
        else:
            print("VAL has no labels → running inference only.")
            _ = pipeline.run_inference(val_split)
    else:
        print("Val split is empty. Skipping evaluation on VAL.")
else:
    print("No VAL files found.")


# --- inference on TEST (usually unlabeled) ---
test_list = dataset.get_split_list("test")
if test_list:
    print("Running inference on TEST split ...")
    test_split = dataset.get_split("test")
    if test_split.get_split_list(): # Check if test split is not empty
        # Check if test files have labels before running test (for metrics)
        try:
            sample = test_split.get_data(0)
            if "label" in sample:
                 print("Test files have labels → running evaluation (OA / per-class IoU / mIoU)")
                 pipeline.run_test() # Use the main pipeline for test evaluation if labels exist
            else:
                 print("Test files do not have labels → running inference-only")
                 _ = pipeline.run_inference(test_split)   # predictions saved under ./logs
                 print("Inference complete.")
        except IndexError:
             print("IndexError: Could not get data from test split. This usually means the dataset did not load any files for the test split.")
        except Exception as e:
             print(f"An error occurred during test split processing: {e}")
    else:
        print("Test split is empty. Skipping inference/evaluation on test split.")
else:
    print("No TEST files found.")

Resolved -> train: 4 val: 0 test: 3
Using device: cpu
CUDA available: False
No VAL files found.
Running inference on TEST split ...


AttributeError: 'ParisLille3DSplit' object has no attribute 'get_split_list'

In [ ]:
# --- inference on TEST (usually unlabeled) ---
test_split = dataset.get_split("test")

if len(test_split) == 0:
    print("Test split is empty.")
else:
    print(f"Running inference on TEST split with {len(test_split)} samples ...")
    # Option A: whole-split inference (saves to ./logs)
    _ = pipeline.run_inference(test_split)

    # Option B: per-sample (handy for debugging)
    # for i in range(len(test_split)):
    #     data = test_split.get_data(i)
    #     result = pipeline.run_inference(data)  # dict with predictions
    #     print(f"Sample {i} -> keys:", list(result.keys()))


Running inference on TEST split with 3 samples ...


TypeError: 'ParisLille3DSplit' object is not subscriptable

In [ ]:
# --- inference on TEST (usually unlabeled) ---
test_split = dataset.get_split("test")

n = len(test_split)
if n == 0:
    print("Test split is empty.")
else:
    print(f"Running per-sample inference on TEST split with {n} samples ...")

    import numpy as np
    import os

    out_dir = "./logs/pl3d_infer"
    os.makedirs(out_dir, exist_ok=True)

    for i in range(n):
        data = test_split.get_data(i)
        # Helpful to see what's inside
        if i == 0:
            print("Sample[0] keys:", list(data.keys()))

        result = pipeline.run_inference(data)
        pred = result.get("predict_labels", None)
        if pred is None:
            # Some versions use "predict_label" (singular) or return as numpy array directly
            pred = result.get("predict_label", result)

        # Try to get a nice base name; fall back to index
        base = (
            os.path.splitext(os.path.basename(data.get("name", "")))[0]
            if isinstance(data.get("name", None), str) and data["name"]
            else f"sample_{i}"
        )
        np.save(os.path.join(out_dir, f"{base}_pred.npy"), np.asarray(pred))
        print(f"Saved predictions -> {os.path.join(out_dir, f'{base}_pred.npy')}")


Running per-sample inference on TEST split with 3 samples ...
Sample[0] keys: ['point', 'feat', 'label']


test 0/1: 100%|██████████| 2901742/2901742 [13:54<00:00, 81.92it/s]/usr/local/lib/python3.11/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:54: RuntimeWarning: Mean of empty slice
  accs.append(np.nanmean(accs))
/usr/local/lib/python3.11/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:87: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(ious))


Saved predictions -> ./logs/pl3d_infer/sample_0_pred.npy



test 0/1: 100%|██████████| 2901742/2901742 [15:23<00:00, 3142.72it/s]

test 0/1: 100%|██████████| 2547609/2547609 [10:57<00:00, 107.18it/s]/usr/local/lib/python3.11/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:54: RuntimeWarning: Mean of empty slice
  accs.append(np.nanmean(accs))
/usr/local/lib/python3.11/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:87: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(ious))


Saved predictions -> ./logs/pl3d_infer/sample_1_pred.npy


test 0/1: 100%|██████████| 2406693/2406693 [10:57<00:00, 156.23it/s]

Saved predictions -> ./logs/pl3d_infer/sample_2_pred.npy


/usr/local/lib/python3.11/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:54: RuntimeWarning: Mean of empty slice
  accs.append(np.nanmean(accs))
/usr/local/lib/python3.11/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:87: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(ious))


In [ ]:
import open3d.ml.torch as ml3d

# Pick which labeled files to evaluate on
eval_files = [
    "training_10_classes/Lille1_1.ply",
    "training_10_classes/Lille1_2.ply",
    "training_10_classes/Lille2.ply",
    "training_10_classes/Paris.ply",
]  # adjust as you like

eval_ds_cfg = dict(cfg.dataset)
eval_ds_cfg["test_files"] = eval_files
eval_ds_cfg["ignored_label_inds"] = [9]   # ignore Unclassified for IoU

eval_ds  = ml3d.datasets.ParisLille3D(**eval_ds_cfg)
eval_pl  = ml3d.pipelines.SemanticSegmentation(model=model, dataset=eval_ds,
                                               device=("cuda" if torch.cuda.is_available() else "cpu"),
                                               **cfg.pipeline)
eval_pl.load_ckpt(CKPT_PATH)
eval_pl.run_test()   # → prints OA, per-class IoU, mIoU



preprocess: 100%|██████████| 3/3 [02:29<00:00, 49.83s/it]

test 0/3: 100%|██████████| 2901742/2901742 [13:23<00:00, 36.20it/s]

test 0/3: 100%|██████████| 2901742/2901742 [14:21<00:00, 3369.17it/s]


test 1/3:   2%|▏         | 40795/2547609 [00:04<04:30, 9252.89it/s]

test 1/3:   2%|▏         | 58518/2547609 [00:08<05:59, 6920.26it/s]

test 1/3:   3%|▎         | 82946/2547609 [00:11<05:52, 6986.69it/s]

test 1/3:   4%|▍         | 96401/2547609 [00:15<07:38, 5340.92it/s]

test 1/3:   5%|▍         | 116917/2547609 [00:19<07:43, 5242.08it/s]

test 1/3:   5%|▌         | 129696/2547609 [00:24<09:15, 4354.42it/s]

test 1/3:   6%|▌         | 150935/2547609 [00:28<08:34, 4659.75it/s]

test 1/3:   7%|▋         | 172561/2547609 [00:32<08:11, 4830.77it/s]

test 1/3:   8%|▊         | 191574/2547609 [00:35<07:49, 5017.60it/s]

test 1/3:   8%|▊         | 205857/2547609 [00:39<08:12, 4754.61it/s]

test 1/3:   9%|▊         | 220268/2547609 [00:43<09:11, 4216.33it/s]

test 1/3:   9%|▉         | 240994

TypeError: 'NoneType' object is not subscriptable